# Tải dữ liệu trên Kaggle

Bộ dữ liệu được nhóm upload trên Kaggle

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
cp kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle\ \(1\).json

chmod: cannot access '/root/.kaggle/kaggle (1).json': No such file or directory


In [ ]:
!kaggle datasets list


ref                                                         title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                         Online Food Dataset                              3KB  2024-03-02 18:50:30          18658        372  0.9411765        
jatinthakur706/most-watched-netflix-original-shows-tv-time  Most watched Netflix original shows (TV Time)    2KB  2024-03-27 09:01:21           1638         26  1.0              
mexwell/drug-consumption-classification                     💊 Drug Consumption Classification               56KB  2024-03-28 15:02:22           1410         31  1.0              
samira1992/diabetes-intermediate-dataset                    🩸 Diabetes - Intermediate 🗃️ Dataset         

In [ ]:
!kaggle datasets download -d phanththutrang/vsl-dataset

 84% 73.0M/87.0M [00:00<00:00, 95.2MB/s]
100% 87.0M/87.0M [00:00<00:00, 94.7MB/s]


In [ ]:
import zipfile

zip_file_path = '/content/vsl-dataset.zip'
extract_to_path = '/content/drive/MyDrive/VietNam SignLanguage'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

# Tăng cường dữ liệu

In [ ]:
!pip install vidaug

In [ ]:
import cv2
import os
import imgaug as ia
import imageio
from imgaug import augmenters as iaa
import shutil
import random
import time
from vidaug import augmentors as va
from multiprocessing import Pool
import concurrent.futures

In [ ]:
no_of_clips_to_augment_per_frame=10
video_clip_names=[]

# Chuẩn hoá dữ liệu

In [ ]:
import os
import cv2
from multiprocessing import Pool
from shutil import copytree

def resize_video(input_output_paths):
    input_path, output_path, target_size = input_output_paths
    cap = cv2.VideoCapture(input_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 30, target_size)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            resized_frame = cv2.resize(frame, target_size)
            out.write(resized_frame)
        else:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

def resize_videos(input_folder, output_folder, target_size):
    # Copy cấu trúc thư mục từ thư mục đầu vào sang thư mục đầu ra
    copytree(input_folder, output_folder)

    # Danh sách các thư mục chứa video trong thư mục đầu vào
    input_subfolders = [f.path for f in os.scandir(input_folder) if f.is_dir()]

    # Duyệt qua từng thư mục con
    for folder in input_subfolders:
        input_subfolder = folder
        output_subfolder = os.path.join(output_folder, os.path.relpath(folder, input_folder))

        # Tạo thư mục đầu ra tương ứng nếu nó không tồn tại
        os.makedirs(output_subfolder, exist_ok=True)

        # Tạo danh sách các cặp đầu vào/đầu ra cho mỗi video trong thư mục con
        input_output_paths = []
        for filename in os.listdir(input_subfolder):
            if filename.endswith(".mp4"):
                input_path = os.path.join(input_subfolder, filename)
                output_path = os.path.join(output_subfolder, filename)
                input_output_paths.append((input_path, output_path, target_size))

        # Sử dụng multiprocessing để resize các video trong thư mục con
        with Pool() as p:
            p.map(resize_video, input_output_paths)


# Đường dẫn đến thư mục chứa video gốc
input_folder = '/content/drive/MyDrive/VietNam SignLanguage/Data'

# Đường dẫn đến thư mục để lưu video đã resize
output_folder = '/content/drive/MyDrive/VietNam SignLanguage/TempOutput'

# Kích thước đích mong muốn (width, height)
target_size = (1280, 720)

# Gọi hàm resize_videos để thực hiện resize cho tất cả các video trong thư mục đầu vào
resize_videos(input_folder, output_folder, target_size)


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/VietNam SignLanguage/TempOutput'

# Tăng cường dữ liệu

In [ ]:

def augment_and_save_frames(video_reader,output_folder_path,video_clip_name,i,fps,w,h):

    temp = video_clip_name.replace(" ","")
    temp = temp.split(".")
    editted_name = temp[0]+"_"+str(i)+"."+temp[1]
    path_of_video_to_save = output_folder_path+"//"+editted_name

    if i%2==0:
        flip=True
    else:
        flip=False
    noise_value = 0

    rotation_angle = random.randint(-30,30)
    print("Rotation angle for augmented clip is ", rotation_angle)
    print("Noise value to add to augmented clip is ", noise_value)

    print(editted_name, rotation_angle, "degrees")
    seq = iaa.Sequential([
        iaa.Fliplr(flip),
        iaa.Affine(rotate=rotation_angle)
    ])
    fourcc = 'mp4v'  # output video codec
    video_writer = cv2.VideoWriter(path_of_video_to_save, cv2.VideoWriter_fourcc(*fourcc),fps,(w,h))

    try:
        while video_reader.isOpened():
            ret, frame = video_reader.read()
            if not ret:
                break
            image_aug = seq(image=frame)
            video_writer.write(image_aug)
    except Exception as e:
        print(e)

    cv2.destroyAllWindows()
    video_reader.release()
    video_writer.release()

In [ ]:
def augment_videos(i):
    try:
        video_path = f"{input_path}//{video_clip_names[clip_no]}"
        video_reader = cv2.VideoCapture(video_path)
        fps = int(video_reader.get(cv2.CAP_PROP_FPS))
        w = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

        print(f"FPS of {video_clip_names[clip_no]} is {fps}")
        start = time.time()
        augment_and_save_frames(video_reader,output_path,video_clip_names[clip_no],i,fps,w,h)
        end = time.time()
        print("Total time taken by single video", end-start)
    except Exception as e:
        print(e)

In [ ]:
input_path = "/content/drive/MyDrive/VietNam SignLanguage/Data/train_dir/Hong"
output_path = "/content/drive/MyDrive/VietNam SignLanguage/Data_Aug_3/Hong"
time_of_code = time.time()
if __name__=='__main__':
    if os.path.exists(output_path) and os.path.isdir(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path,exist_ok=True)

    video_clip_names = os.listdir(input_path)
    print(f"Videos found are {video_clip_names}")
    no_of_clips_available = len(video_clip_names)

    for clip_no in range(no_of_clips_available):
        print("No. of videos to be augmented per input", no_of_clips_to_augment_per_frame)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(augment_videos, list(range(no_of_clips_to_augment_per_frame)))



    end_time = time.time()
    print("Full time by code", end_time-time_of_code)

Videos found are ['Hong_001_001.mp4', 'Hong_001_002.mp4', 'Hong_001_004.mp4', 'Hong_001_005.mp4', 'Hong_001_003.mp4', 'Hong_002_001.mp4', 'Hong_002_003.mp4', 'Hong_002_002.mp4', 'Hong_002_005.mp4', 'Hong_002_004.mp4', 'Hong_003_001.mp4', 'Hong_003_003.mp4', 'Hong_003_004.mp4', 'Hong_003_002.mp4', 'Hong_003_005.mp4', 'Hong_004_001.mp4', 'Hong_004_002.mp4', 'Hong_004_004.mp4', 'Hong_004_003.mp4', 'Hong_004_005.mp4', 'Hong_005_001.mp4', 'Hong_005_002.mp4', 'Hong_005_003.mp4', 'Hong_006_002.mp4', 'Hong_006_001.mp4', 'Hong_006_003.mp4', 'Hong_005_004.mp4', 'Hong_005_005.mp4', 'Hong_007_001.mp4', 'Hong_006_005.mp4', 'Hong_006_004.mp4', 'Hong_007_002.mp4', 'Hong_007_003.mp4', 'Hong_008_001.mp4', 'Hong_007_005.mp4', 'Hong_007_004.mp4', 'Hong_008_002.mp4', 'Hong_008_004.mp4', 'Hong_008_003.mp4', 'Hong_009_003.mp4', 'Hong_009_002.mp4', 'Hong_008_005.mp4', 'Hong_009_004.mp4', 'Hong_009_001.mp4', 'Hong_010_002.mp4', 'Hong_010_001.mp4', 'Hong_009_005.mp4', 'Hong_010_003.mp4', 'Hong_010_005.mp4', 'H

In [ ]:
input_path = "/content/drive/MyDrive/VietNam SignLanguage/Data/train_dir/Vang"
output_path = "/content/drive/MyDrive/VietNam SignLanguage/Data_Aug_3/Vang"
time_of_code = time.time()
if __name__=='__main__':
    if os.path.exists(output_path) and os.path.isdir(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path,exist_ok=True)

    video_clip_names = os.listdir(input_path)
    print(f"Videos found are {video_clip_names}")
    no_of_clips_available = len(video_clip_names)

    for clip_no in range(no_of_clips_available):
        print("No. of videos to be augmented per input", no_of_clips_to_augment_per_frame)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(augment_videos, list(range(no_of_clips_to_augment_per_frame)))



    end_time = time.time()
    print("Full time by code", end_time-time_of_code)

Videos found are ['Vang_001_001.mp4', 'Vang_001_003.mp4', 'Vang_001_004.mp4', 'Vang_001_002.mp4', 'Vang_001_005.mp4', 'Vang_002_002.mp4', 'Vang_002_003.mp4', 'Vang_002_001.mp4', 'Vang_002_004.mp4', 'Vang_003_001.mp4', 'Vang_002_005.mp4', 'Vang_003_002.mp4', 'Vang_003_003.mp4', 'Vang_003_005.mp4', 'Vang_004_001.mp4', 'Vang_003_004.mp4', 'Vang_004_003.mp4', 'Vang_004_005.mp4', 'Vang_005_001.mp4', 'Vang_005_002.mp4', 'Vang_005_004.mp4', 'Vang_005_005.mp4', 'Vang_006_002.mp4', 'Vang_006_001.mp4', 'Vang_004_004.mp4', 'Vang_004_002.mp4', 'Vang_006_003.mp4', 'Vang_006_005.mp4', 'Vang_007_002.mp4', 'Vang_007_001.mp4', 'Vang_006_004.mp4', 'Vang_007_004.mp4', 'Vang_007_003.mp4', 'Vang_007_005.mp4', 'Vang_008_001.mp4', 'Vang_008_003.mp4', 'Vang_008_002.mp4', 'Vang_008_004.mp4', 'Vang_008_005.mp4', 'Vang_009_001.mp4', 'Vang_009_002.mp4', 'Vang_009_003.mp4', 'Vang_009_004.mp4', 'Vang_010_001.mp4', 'Vang_009_005.mp4', 'Vang_010_002.mp4', 'Vang_005_003.mp4', 'Vang_010_003.mp4', 'Vang_010_004.mp4', 'V

In [ ]:
input_path = "/content/drive/MyDrive/VietNam SignLanguage/Data/train_dir/Do"
output_path = "/content/drive/MyDrive/VietNam SignLanguage/Data_Aug_3/Do"
time_of_code = time.time()
if __name__=='__main__':
    if os.path.exists(output_path) and os.path.isdir(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path,exist_ok=True)

    video_clip_names = os.listdir(input_path)
    print(f"Videos found are {video_clip_names}")
    no_of_clips_available = len(video_clip_names)

    for clip_no in range(no_of_clips_available):
        print("No. of videos to be augmented per input", no_of_clips_to_augment_per_frame)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(augment_videos, list(range(no_of_clips_to_augment_per_frame)))



    end_time = time.time()
    print("Full time by code", end_time-time_of_code)

Videos found are ['Do_001_001.mp4', 'Do_001_002.mp4', 'Do_001_003.mp4', 'Do_001_005.mp4', 'Do_001_004.mp4', 'Do_002_001.mp4', 'Do_002_004.mp4', 'Do_002_002.mp4', 'Do_002_003.mp4', 'Do_003_001.mp4', 'Do_002_005.mp4', 'Do_003_002.mp4', 'Do_003_004.mp4', 'Do_003_005.mp4', 'Do_004_001.mp4', 'Do_004_002.mp4', 'Do_004_003.mp4', 'Do_004_005.mp4', 'Do_004_004.mp4', 'Do_005_001.mp4', 'Do_005_002.mp4', 'Do_003_003.mp4', 'Do_005_003.mp4', 'Do_005_004.mp4', 'Do_005_005.mp4', 'Do_006_002.mp4', 'Do_006_001.mp4', 'Do_006_004.mp4', 'Do_006_003.mp4', 'Do_006_005.mp4', 'Do_007_002.mp4', 'Do_007_001.mp4', 'Do_007_003.mp4', 'Do_008_001.mp4', 'Do_007_004.mp4', 'Do_007_005.mp4', 'Do_008_005.mp4', 'Do_008_002.mp4', 'Do_008_003.mp4', 'Do_008_004.mp4', 'Do_009_001.mp4', 'Do_009_003.mp4', 'Do_009_002.mp4', 'Do_009_004.mp4', 'Do_009_005.mp4', 'Do_010_001.mp4', 'Do_010_004.mp4', 'Do_010_003.mp4', 'Do_010_005.mp4', 'Do_010_002.mp4']
No. of videos to be augmented per input 3
FPS of Do_001_001.mp4 is 59
Rotation ang

In [ ]:
input_path = "/content/drive/MyDrive/VietNam SignLanguage/Data/train_dir/Xanh duong"
output_path = "/content/drive/MyDrive/VietNam SignLanguage/Data_Aug_3/Xanh duong"
time_of_code = time.time()
if __name__=='__main__':
    if os.path.exists(output_path) and os.path.isdir(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path,exist_ok=True)

    video_clip_names = os.listdir(input_path)
    print(f"Videos found are {video_clip_names}")
    no_of_clips_available = len(video_clip_names)

    for clip_no in range(no_of_clips_available):
        print("No. of videos to be augmented per input", no_of_clips_to_augment_per_frame)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(augment_videos, list(range(no_of_clips_to_augment_per_frame)))



    end_time = time.time()
    print("Full time by code", end_time-time_of_code)

Videos found are ['Xanhduong_001_001.mp4', 'Xanhduong_001_003.mp4', 'Xanhduong_001_002.mp4', 'Xanhduong_001_004.mp4', 'Xanhduong_002_002.mp4', 'Xanhduong_001_005.mp4', 'Xanhduong_002_003.mp4', 'Xanhduong_002_004.mp4', 'Xanhduong_002_005.mp4', 'Xanhduong_003_001.mp4', 'Xanhduong_002_001.mp4', 'Xanhduong_003_002.mp4', 'Xanhduong_003_003.mp4', 'Xanhduong_003_004.mp4', 'Xanhduong_003_005.mp4', 'Xanhduong_004_001.mp4', 'Xanhduong_004_002.mp4', 'Xanhduong_004_003.mp4', 'Xanhduong_005_001.mp4', 'Xanhduong_004_005.mp4', 'Xanhduong_004_004.mp4', 'Xanhduong_005_002.mp4', 'Xanhduong_005_003.mp4', 'Xanhduong_005_004.mp4', 'Xanhduong_005_005.mp4', 'Xanhduong_006_001.mp4', 'Xanhduong_006_003.mp4', 'Xanhduong_006_002.mp4', 'Xanhduong_006_004.mp4', 'Xanhduong_006_005.mp4', 'Xanhduong_007_002.mp4', 'Xanhduong_007_001.mp4', 'Xanhduong_007_003.mp4', 'Xanhduong_007_004.mp4', 'Xanhduong_008_001.mp4', 'Xanhduong_008_002.mp4', 'Xanhduong_008_003.mp4', 'Xanhduong_008_004.mp4', 'Xanhduong_007_005.mp4', 'Xanhdu

In [ ]:
input_path = "/content/drive/MyDrive/VietNam SignLanguage/Data/train_dir/Xanh la"
output_path = "/content/drive/MyDrive/VietNam SignLanguage/Data_Aug_3/Xanh la"
time_of_code = time.time()
if __name__=='__main__':
    if os.path.exists(output_path) and os.path.isdir(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path,exist_ok=True)

    video_clip_names = os.listdir(input_path)
    print(f"Videos found are {video_clip_names}")
    no_of_clips_available = len(video_clip_names)

    for clip_no in range(no_of_clips_available):
        print("No. of videos to be augmented per input", no_of_clips_to_augment_per_frame)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(augment_videos, list(range(no_of_clips_to_augment_per_frame)))



    end_time = time.time()
    print("Full time by code", end_time-time_of_code)

Videos found are ['Xanhla_001_002.mp4', 'Xanhla_001_003.mp4', 'Xanhla_001_005.mp4', 'Xanhla_001_001.mp4', 'Xanhla_001_004.mp4', 'Xanhla_002_001.mp4', 'Xanhla_002_002.mp4', 'Xanhla_002_004.mp4', 'Xanhla_002_005.mp4', 'Xanhla_003_002.mp4', 'Xanhla_003_003.mp4', 'Xanhla_003_005.mp4', 'Xanhla_003_004.mp4', 'Xanhla_004_001.mp4', 'Xanhla_004_002.mp4', 'Xanhla_005_001.mp4', 'Xanhla_004_004.mp4', 'Xanhla_004_003.mp4', 'Xanhla_005_002.mp4', 'Xanhla_002_003.mp4', 'Xanhla_003_001.mp4', 'Xanhla_005_003.mp4', 'Xanhla_005_004.mp4', 'Xanhla_005_005.mp4', 'Xanhla_006_002.mp4', 'Xanhla_006_001.mp4', 'Xanhla_006_003.mp4', 'Xanhla_006_004.mp4', 'Xanhla_006_005.mp4', 'Xanhla_007_001.mp4', 'Xanhla_004_005.mp4', 'Xanhla_007_002.mp4', 'Xanhla_007_003.mp4', 'Xanhla_007_005.mp4', 'Xanhla_008_001.mp4', 'Xanhla_008_002.mp4', 'Xanhla_008_003.mp4', 'Xanhla_009_001.mp4', 'Xanhla_009_002.mp4', 'Xanhla_007_004.mp4', 'Xanhla_009_004.mp4', 'Xanhla_009_005.mp4', 'Xanhla_010_002.mp4', 'Xanhla_010_001.mp4', 'Xanhla_008_00

# Phân chia dữ liệu

In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil

input_dir = '/content/drive/MyDrive/VietNam SignLanguage/Data_Aug_3'
output_dir = '/content/drive/MyDrive/VietNam SignLanguage/train_test'

train_dir = os.path.join(output_dir, "train_dir_2")
# test_dir = os.path.join(output_dir, 'test_dir')

os.makedirs(train_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)

for folder in os.listdir(input_dir):
    folder_samp = os.path.join(input_dir, folder)
    folder_train = os.path.join(train_dir, folder)
    # folder_test = os.path.join(test_dir, folder)

    os.makedirs(folder_train, exist_ok=True)
    # os.makedirs(folder_test, exist_ok=True)

    for file in os.listdir(folder_samp):
        # if file.endswith("4.mp4"):
        #     shutil.copy(os.path.join(folder_samp, file), os.path.join(folder_test, file))
        # else:
        shutil.copy(os.path.join(folder_samp, file), os.path.join(folder_train, file))

print("Đã chia tập train thành công!")


Đã chia tập train thành công!
